In [9]:
import sys
sys.path.append("/planning_through_contact")

In [10]:
import numpy as np
import pandas as pd
from examples.planar_hand.contact_sampler import PlanarHandContactSampler

from qsim.parser import QuasistaticParser

from irs_rrt.irs_rrt import IrsNode
from irs_rrt.irs_rrt_projection import IrsRrtProjection
from irs_rrt.rrt_params import IrsRrtProjectionParams, SmoothingMode
from examples.planar_hand.planar_hand_setup import *

from irs_mpc2.quasistatic_visualizer import QuasistaticVisualizer


In [11]:

# %% quasistatic system
q_parser = QuasistaticParser(q_model_path)

q_vis = QuasistaticVisualizer.make_visualizer(q_parser)
q_sim, q_sim_py = q_vis.q_sim, q_vis.q_sim_py
plant = q_sim.get_plant()

dim_x = q_sim.num_dofs()
dim_u = q_sim.num_actuated_dofs()
idx_a_l = plant.GetModelInstanceByName(robot_l_name)
idx_a_r = plant.GetModelInstanceByName(robot_r_name)
idx_u = plant.GetModelInstanceByName(object_name)
contact_sampler = PlanarHandContactSampler(
    q_sim=q_sim, q_sim_py=q_sim_py, pinch_prob=0.1
)

q_u0 = np.array([0.0, 0.35, 0])
q0_dict = contact_sampler.calc_enveloping_grasp(q_u0)
q0 = q_sim.get_q_vec_from_dict(q0_dict)


joint_limits = {
    idx_u: np.array([[-0.3, 0.3], [0.3, 0.5], [-0.01, np.pi + 0.01]])
}

INFO:drake:Meshcat listening for connections at http://localhost:7002


In [12]:

# %% RRT testing
rrt_params = IrsRrtProjectionParams(q_model_path, joint_limits)
rrt_params.h = h
rrt_params.smoothing_mode = SmoothingMode.k1AnalyticPyramid
rrt_params.log_barrier_weight_for_bundling = 100
rrt_params.root_node = IrsNode(q0)
rrt_params.max_size = 1000
rrt_params.goal = np.copy(q0)
rrt_params.goal[2] = np.pi
rrt_params.termination_tolerance = 0.01
rrt_params.goal_as_subgoal_prob = 0.3
rrt_params.regularization = 1e-4
rrt_params.rewire = False
rrt_params.grasp_prob = 0.
rrt_params.distance_threshold = np.inf
rrt_params.stepsize = 0.35
rrt_params.distance_metric = "local_u"

# params.distance_metric = 'global'  # If using global metric
rrt_params.global_metric = np.array([10.0, 10.0, 1.0, 0.1, 0.1, 0.1, 0.1])

print("start state:", q0)
print("goal state:", rrt_params.goal)

start state: [ 0.          0.35        0.         -0.88050497 -0.60592241  0.88050497
  0.60592241]
goal state: [ 0.          0.35        3.14159265 -0.88050497 -0.60592241  0.88050497
  0.60592241]


In [13]:
rrt_params.use_free_solvers = True
# contact_sampler.sim_params.use_free_solvers = True

In [14]:

prob_rrt = IrsRrtProjection(rrt_params, contact_sampler, q_sim, q_sim_py)
prob_rrt.iterate()

100%|█████████▉| 999/1000 [00:48<00:00, 20.64it/s]


In [16]:
(
    q_knots_trimmed,
    u_knots_trimmed,
) = prob_rrt.get_trimmed_q_and_u_knots_to_goal()
q_vis.publish_trajectory(q_knots_trimmed, h=rrt_params.h)

closest distance to subgoal 230.32616681928008


In [17]:
d_batch = prob_rrt.calc_distance_batch(rrt_params.goal)
print("minimum distance: ", d_batch.min())

# %%
node_id_closest = np.argmin(d_batch)

# %%
prob_rrt.save_tree(f"planar_hand_tree_{rrt_params.max_size}_{0}.pkl")


# %%
closest_state = prob_rrt.q_matrix[node_id_closest, :]
print("closest state", closest_state)
print("goal state", rrt_params.goal)


minimum distance:  230.32616681928008
closest state [-0.03975248  0.3758021   0.38605703 -0.33596209 -1.49552414  1.07263472
 -0.09972964]
goal state [ 0.          0.35        3.14159265 -0.88050497 -0.60592241  0.88050497
  0.60592241]


In [20]:
def compute_angular_error(angle, goal_angle):
    delta = 360 * (goal_angle - angle) / (2 * np.pi)
    delta_wraped = delta - (delta // 360) * 360
    if delta_wraped > 180:
        delta_wraped -= 360
    elif delta_wraped < -180:
        delta_wraped += 360
    angular_error = np.abs(delta_wraped)
    return angular_error

def compute_minimal_angular_error(prob_rrt):
    rrt_params = prob_rrt.rrt_params
    max_size = rrt_params.max_size
    goal_angle = rrt_params.goal[2]
    angular_errors = np.array([compute_angular_error(prob_rrt.q_matrix[i,2], goal_angle) for i in range(max_size)])
    return np.min(angular_errors)


def compute_closest_node_angular_error(prob_rrt):
    rrt_params = prob_rrt.rrt_params
    d_batch = prob_rrt.calc_distance_batch(rrt_params.goal)
    node_id_closest = np.argmin(d_batch)
    goal_angle = rrt_params.goal[2]
    angular_error = compute_angular_error(prob_rrt.q_matrix[node_id_closest,2], goal_angle)
    return angular_error


closest_node_angular_error = compute_closest_node_angular_error(prob_rrt)
minimal_angular_error = compute_minimal_angular_error(prob_rrt)

print("closest node angular error", closest_node_angular_error)
print("minimal angular error", minimal_angular_error)

angular error 157.88056157996837
closest node angular error 157.88056157996837
minimal angular error 146.63380456329722


In [ ]:
365.0 // 360


1.0